# Práctica 2: Limpieza y análisis de datos
#### Tipología y ciclo de vida de los datos
### Marina Peña, Martín Sanchez

## Descripción del dataset

## Integración y selección de los datos

## Limpieza de los datos

### Ceros y elementos vacíos

### Valores extremos

## Análisis de los datos

### Selección de grupos de datos

### Comprobación de normalidad y homogeneidad de la varianza

### Pruebas estadísticas

## Conclusiones